# Consigna
+ def **userdata( *`User_id` : str* )**:
    Debe devolver `cantidad` de dinero gastado por el usuario, el `porcentaje` de recomendación en base a reviews.recommend y `cantidad de items`.
    
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

Para Resolver esta voy a trabajar Tanto con el Dataframe de Reviews y el de Games

In [1]:
import pandas as pd

In [42]:
games_path = '../ETL/Data_Extracted/DataFrame_games.parquet'
GamesDF = pd.read_parquet(games_path)

In [43]:
reviews_path = '../ETL/Data_Extracted/DataFrame_reviews.parquet'
ReviewsDF = pd.read_parquet(reviews_path)

In [45]:
ReviewsDF.head(1)

,user_id,recommend,item_id
0,76561197970982479,True,1250


In [44]:
#Filtro  por solo las columnas que  voy a usar
#En este caso me quedo con el id y el precio
GamesDF = GamesDF[['id','price']]
ReviewsDF = ReviewsDF[['user_id','recommend','item_id']]

In [14]:
#Ahora tengo dos Dataframes: reviewDF y GamesDF la idea es unirlos en uno solo
#Cambio el nombre para poder unirlos por esa columna en comun
GamesDF = GamesDF.rename(columns={'id': 'item_id'})


In [15]:
GamesDF

,item_id,price
0,761140,4.99
1,643980,0.00
2,670290,0.00
3,767400,0.99
4,772540,3.99
...,...,...
27457,745400,1.99
27458,773640,1.99
27459,733530,4.99
27460,610660,1.99


Debo trabajar solo con numeros en la columna de precio, si algo sale mal al tratar de convertir lo a un formato numerico, le asigno 0

In [46]:
# Convertir la columna 'precio' a tipo numérico y reemplazar los valores no numéricos con 0 que vendrian a ser los free to play
GamesDF['price'] = pd.to_numeric(GamesDF['price'], errors='coerce').fillna(0)

In [47]:
GamesDF['price'].head(25)

0      4.99
1      0.00
2      0.00
3      0.99
4      3.99
5      9.99
6     18.99
7     29.99
8     10.99
9      3.99
10     2.99
11     1.59
12    14.99
13     1.99
14    59.99
15     0.99
16     2.99
17     2.99
18     4.99
19     9.99
20     9.99
21     1.99
22     0.99
23     8.99
24     0.99
Name: price, dtype: float64

In [48]:
ReviewsDF

,user_id,recommend,item_id
0,76561197970982479,True,1250
1,76561197970982479,True,22200
2,76561197970982479,True,43110
3,js41637,True,251610
4,js41637,True,227300
...,...,...,...
59300,76561198312638244,True,70
59301,76561198312638244,True,362890
59302,LydiaMorley,True,273110
59303,LydiaMorley,True,730


In [49]:
#Ahora tengo dos Dataframes: reviewDF y GamesDF la idea es unirlos en uno solo
#Cambio el nombre para poder unirlos por esa columna en comun
GamesDF = GamesDF.rename(columns={'id': 'item_id'})

In [50]:
#Agrego el precio a la informacion de mis usuarios
UserDataDF = pd.merge(GamesDF, ReviewsDF, on='item_id' , how='left')

In [51]:
UserDataDF

,item_id,price,user_id,recommend
0,761140,4.99,NaN,NaN
1,643980,0.00,NaN,NaN
2,670290,0.00,NaN,NaN
3,767400,0.99,NaN,NaN
4,772540,3.99,NaN,NaN
...,...,...,...,...
74102,745400,1.99,NaN,NaN
74103,773640,1.99,NaN,NaN
74104,733530,4.99,NaN,NaN
74105,610660,1.99,NaN,NaN


In [52]:
# reconstruyo el user id
UserDataDF['user_id'] = ReviewsDF['user_id']

In [53]:
# reconstruyo el user id
UserDataDF['recommend'] = ReviewsDF['recommend']

In [54]:
UserDataDF

,item_id,price,user_id,recommend
0,761140,4.99,76561197970982479,True
1,643980,0.00,76561197970982479,True
2,670290,0.00,76561197970982479,True
3,767400,0.99,js41637,True
4,772540,3.99,js41637,True
...,...,...,...,...
74102,745400,1.99,NaN,NaN
74103,773640,1.99,NaN,NaN
74104,733530,4.99,NaN,NaN
74105,610660,1.99,NaN,NaN


Es normal que tenga muchos Nan en User_id ya que no todos han opinado ("No importa si a favor o encontra")

In [55]:
UserDataDF['user_id'].isna().sum()

14802

In [56]:
UserDataDF_Opinaron = UserDataDF.dropna()

In [57]:
UserDataDF_Opinaron

,item_id,price,user_id,recommend
0,761140,4.99,76561197970982479,True
1,643980,0.00,76561197970982479,True
2,670290,0.00,76561197970982479,True
3,767400,0.99,js41637,True
4,772540,3.99,js41637,True
...,...,...,...,...
59300,212070,0.00,76561198312638244,True
59301,222281,49.99,76561198312638244,True
59302,222282,79.99,LydiaMorley,True
59303,222280,19.99,LydiaMorley,True


Para simplificar, voy a cambiar la variable Booleano por un bit

In [58]:
UserDataDF_Opinaron['recommend'] = UserDataDF_Opinaron['recommend'].replace({True: 1, False: 0})

C:\Users\ahurt\AppData\Local\Temp\ipykernel_20184\3631043485.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  UserDataDF_Opinaron['recommend'] = UserDataDF_Opinaron['recommend'].replace({True: 1, False: 0})
C:\Users\ahurt\AppData\Local\Temp\ipykernel_20184\3631043485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UserDataDF_Opinaron['recommend'] = UserDataDF_Opinaron['recommend'].replace({True: 1, False: 0})


In [59]:
UserDataDF_Opinaron

,item_id,price,user_id,recommend
0,761140,4.99,76561197970982479,1
1,643980,0.00,76561197970982479,1
2,670290,0.00,76561197970982479,1
3,767400,0.99,js41637,1
4,772540,3.99,js41637,1
...,...,...,...,...
59300,212070,0.00,76561198312638244,1
59301,222281,49.99,76561198312638244,1
59302,222282,79.99,LydiaMorley,1
59303,222280,19.99,LydiaMorley,1


In [60]:
def userdata(user_id):
    # Filtrar el DataFrame según el user_id
    user_data = UserDataDF[UserDataDF['user_id'] == user_id]

    # Cantidad de dinero gastado por el usuario
    dinero_gastado = user_data['price'].sum()

    # Porcentaje de recomendación
    total_recomendaciones = user_data['recommend'].sum()
    porcentaje_recomendacion = (total_recomendaciones / len(user_data)) * 100

    # Cantidad de items
    cantidad_items = len(user_data)

    # Formatear los resultados
    resultado = {
        "Usuario": user_id,
        "Dinero gastado": f"{dinero_gastado} USD",
        "% de recomendación": f"{porcentaje_recomendacion:.2f}%",
        "Cantidad de items": cantidad_items
    }

    return resultado

In [61]:
#prueba
print(userdata('js41637'))

{'Usuario': 'js41637', 'Dinero gastado': '14.97 USD', '% de recomendación': '100.00%', 'Cantidad de items': 3}


In [62]:
# Genero el archivo para hacer las consultas en la api
UserDataDF_Opinaron.to_parquet('./Out/DataFrame-punto02_UserData.parquet', index=False)